In [3]:
import sys, os
sys.path.append(os.path.dirname(os.getcwd()))

import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.express as px
from jupyter_dash import JupyterDash
import pandas as pd

from pof.failure_mode import FailureMode

In [4]:
fm = FailureMode().set_default()

impact = fm.tasks['ocr'].impacts()['condition']['wall_thickness']
prefix=''

In [15]:
c_name = dbc.InputGroup(
    [
        dbc.InputGroupAddon(dbc.Checkbox(), addon_type="prepend"),
        dbc.Label("Cond Name", className="mr-2"),

    ]
)

target = dbc.FormGroup(
    [
        dbc.Label("Target", className="mr-2"),
        dbc.Input(
            type="number",
            id= prefix + "target",
            value=impact['target'],
        ),
    ],
    className="mr-3",
)

method = dbc.FormGroup(
    [
        dbc.Label("Method", className="mr-2"),
        dbc.Select(
            id=prefix + "method_dropdown",
            options=[{'label' : method, 'value' : method} for method in ['reduction_factor', 'tbc']],
            value=impact['method'],
        ),
    ],
    className="mr-3",
)

axis = dbc.FormGroup(
    [
        dbc.Label("Axis", className="mr-2"),
        dbc.Select(
            id=prefix + "axis_dropdown",
            options=[{'label' : axis, 'value' : axis} for axis in ['condition', 'time']],
            value=impact['axis'],
        ),
    ],
    className="mr-3",
)

form = dbc.Form([c_name, target, method, axis], inline=True, className="dash-bootstrap")

In [28]:
trigger_card = dbc.Col(
    dbc.Card(
        [
            dbc.CardHeader("Impact Trigger"),
            dbc.CardBody(
                    [
                        form,
                    ],
            ),
            dbc.CardBody(
                    [
                        "asdf",
                    ],
            ),
        ]
    )
)
        

input_group2 = html.Div(
    [
        dbc.InputGroup(
            [
                dbc.InputGroupAddon(dbc.Checkbox(), addon_type="prepend"),
                dbc.InputGroupAddon(dbc.RadioButton(), addon_type="prepend"),
            ],
            className="mb-3",
        ),
        dbc.InputGroup(
            [
                dbc.InputGroupAddon(dbc.Checkbox(), addon_type="prepend"),

                dbc.Button(
                    "Task Name",
                    color="link",
                    id="collapse-button",
                ),
                dbc.Col(
                    [
                        dbc.Label("Axis", className="mr-2"),
                        dbc.Collapse(
                            dbc.Card(dbc.CardBody(
                                [
                                    dbc.Col(dbc.Card(dbc.CardBody())),
                                    trigger_card, 
                                    dbc.Col(dbc.Card(dbc.CardBody())),
                                ]
                            )),
                            id="collapse",
                            is_open=True
                        ),
                    ]
                )

            ]
        ),

    ]
)

In [29]:

# Build App
external_stylesheets = [dbc.themes.BOOTSTRAP]
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Layout
app.layout = input_group2

@app.callback(
    Output("collapse", "is_open"),
    [Input("collapse-button", "n_clicks")],
    [State("collapse", "is_open")],
)
def toggle_collapse(n, is_open):
    if n:
        return not is_open
    return is_open

#Execute
app.run_server(mode='inline')